In [1]:
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

In [2]:
from Util import get_openai_api_key
OPEN_API_KEY= get_openai_api_key()

In [3]:
llm_config={
    "config_list":{
        "model":"gpt-3.5-turbo",
        "api_key":OPEN_API_KEY
    }
}

In [31]:
system_assistant=AssistantAgent(
    name="system_assistant",
    human_input_mode="NEVER",
    llm_config=llm_config,
    code_execution_config={
        "work_dir":"50_Startups.csv",
        "use_docker":False
    },
    system_message="""You are a data analysis assistant. Given a dataset containing a 'Profit' column, your task is to sort the dataset in descending order of profit and display the top 10 rows with the highest profit values as output. Ensure the output includes the entire row for each of the top 10 entries."""
)

In [30]:
import pandas as pd
df= pd.read_csv("50_Startups.csv")
docs_content= df.sort_values(by="Profit",ascending=False).head(10)

In [19]:
docs_content

,R&D Spend,Administration,Marketing Spend,State,Profit
49,0.00,116983.80,45173.06,California,14681.40
48,542.05,51743.15,0.00,New York,35673.41
47,0.00,135426.92,0.00,California,42559.73
46,1315.46,115816.21,297114.46,Florida,49490.75
45,1000.23,124153.04,1903.93,New York,64926.08
44,22177.74,154806.14,28334.72,California,65200.33
43,15505.73,127382.30,35534.17,New York,69758.98
42,23640.93,96189.63,148001.11,California,71498.49
41,27892.92,84710.77,164470.71,Florida,77798.83
40,28754.33,118546.05,172795.67,California,78239.91


In [32]:
from autogen import UserProxyAgent

user_agent= UserProxyAgent(
    name="user_agent",
    code_execution_config={
        "work_dir":"50_Startups.csv",
        "use_docker":False
    },
    system_message="You are a retriever agent that provides access to external CSV files like 50_Startups.csv"
)

In [33]:
from autogen import GroupChat,GroupChatManager
group_chat= GroupChat(agents=[system_assistant,user_agent],messages=[],max_round=5)
manager=GroupChatManager(groupchat=group_chat,llm_config=llm_config)

In [34]:
user_agent.initiate_chat(manager,message="Here is the dataset containing the 'Profit' column.",code="""
import pandas as pd
df = pd.read_csv('50_Startups.csv')
df.sort_values(by='Profit', ascending=True).head(10)
""")

user_agent (to chat_manager):

Here is the dataset containing the 'Profit' column.

--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.



Next speaker: system_assistant

system_assistant (to chat_manager):

Please provide the dataset so we can proceed with sorting it in descending order of profit and displaying the top 10 rows with the highest profit values.

--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.



Next speaker: system_assistant

system_assistant (to chat_manager):

Could you please provide the dataset containing the 'Profit' column so that I can assist you further?

--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.



Next speaker: system_assistant

system_assistant (to chat_manager):

To analyze the data, I would need the dataset with the 'Profit' column.

--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.



Next speaker: system_assistant

system_assistant (to chat_manager):

Once you provide the dataset, I will be able to sort it in descending order of profit and display the top 10 rows with the highest profit values.

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (ff059988-5d42-43ca-b23f-2dc4aa342961): Maximum rounds (5) reached


ChatResult(chat_id=None, chat_history=[{'content': "Here is the dataset containing the 'Profit' column.", 'role': 'assistant', 'name': 'user_agent'}, {'content': 'Please provide the dataset so we can proceed with sorting it in descending order of profit and displaying the top 10 rows with the highest profit values.', 'name': 'system_assistant', 'role': 'user'}, {'content': "Could you please provide the dataset containing the 'Profit' column so that I can assist you further?", 'name': 'system_assistant', 'role': 'user'}, {'content': "To analyze the data, I would need the dataset with the 'Profit' column.", 'name': 'system_assistant', 'role': 'user'}, {'content': 'Once you provide the dataset, I will be able to sort it in descending order of profit and display the top 10 rows with the highest profit values.', 'name': 'system_assistant', 'role': 'user'}], summary='Once you provide the dataset, I will be able to sort it in descending order of profit and display the top 10 rows with the hig

In [27]:
#print(chatResult)

In [21]:
system_assistant.reset()
professor= "Can you display top 10 profit data"
results= retrieve_agent.initiate_chat(system_assistant,message=retrieve_agent.message_generator,problem=professor)

File 50_Startups.csv.csv does not exist. Skipping.
2025-06-30 14:23:25,934 - autogen.agentchat.contrib.retrieve_user_proxy_agent - INFO - Found 0 chunks.


Trying to create collection.
VectorDB returns doc_ids:  [[]]
No more context, will terminate.
retrieve_agent (to system_assistant):

TERMINATE

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (fac2ceee-b72f-4a8d-bd90-d1a67ed8e01b): Termination message condition on agent 'system_assistant' met
